# Chessboard Puzzle

## Imports

In [37]:
# IMPORTS

import numpy as np
from datascience import *
import math
import pandas as pd
import random

import matplotlib.pyplot as plots
import matplotlib
matplotlib.use('Agg', warn=False)
%matplotlib inline
plots.style.use('fivethirtyeight')

## The Question

- There are two prisoners and a warden
- The warden will let both prisoners free if they can solve the puzzle
- We are prisoner 1, sitting with the warden, and prisoner 2 is not here
- The warden shows us a Chess board with 1 coin on each of the 64 squares, flipped randomly heads or tails
- He then puts a key under one of the squares (secret compartment in each square)
- We know which square it is

- We have to flip exactly ONE coin
- So that prisoner 2, when he returns, can know exactly where the key is
- Before the key is placed under the square, prisoners 1 and 2 were able to come up with a plan together
- The warden knows the plan, and can flip the coins in any way he wants to mess up our plan

- How is this possible?

## The Board

In [76]:
# MAKE 64 RANDOM COIN FLIPS, 0=HEADS, 1=TAILS

choices = []
for i in np.arange(64):
    choice = random.choice([0,1])
    choices.append(choice)
choices = np.array(choices)

In [77]:
# RESHAPE DATA TO 8 BY 8 CHESS BOARD

board = choices.reshape(8,8)
board = pd.DataFrame(board)
board

,0,1,2,3,4,5,6,7
0,1,0,1,1,1,0,0,1
1,0,0,1,0,0,0,0,1
2,1,0,1,1,1,0,0,0
3,1,0,1,1,1,0,0,0
4,1,1,1,0,0,0,0,0
5,1,0,0,1,0,0,1,1
6,0,1,0,0,1,1,0,1
7,1,0,0,1,1,1,0,0


In [74]:
# CONVERT EACH PLACE ON BOARD TO NUMBERS 0 through 63

numbered = np.arange(64).reshape(8,8)
numbered = pd.DataFrame(numbered)
numbered

,0,1,2,3,4,5,6,7
0,0,1,2,3,4,5,6,7
1,8,9,10,11,12,13,14,15
2,16,17,18,19,20,21,22,23
3,24,25,26,27,28,29,30,31
4,32,33,34,35,36,37,38,39
5,40,41,42,43,44,45,46,47
6,48,49,50,51,52,53,54,55
7,56,57,58,59,60,61,62,63


In [73]:
# CONVERT NUMBERS TO BINARY

binaries = []
for i in np.arange(64):
    binary = bin(i)[2:]
    binaries.append(binary)
binaries = np.array(binaries).reshape(8,8)
binary_board = pd.DataFrame(binaries)
binary_board

,0,1,2,3,4,5,6,7
0,0,1,10,11,100,101,110,111
1,1000,1001,1010,1011,1100,1101,1110,1111
2,10000,10001,10010,10011,10100,10101,10110,10111
3,11000,11001,11010,11011,11100,11101,11110,11111
4,100000,100001,100010,100011,100100,100101,100110,100111
5,101000,101001,101010,101011,101100,101101,101110,101111
6,110000,110001,110010,110011,110100,110101,110110,110111
7,111000,111001,111010,111011,111100,111101,111110,111111


## The Strategies

### Number 1: Mod Sum

In [97]:
# TAKE DOT PRODUCT OF CHESS BOARD AND NUMBERED (0-63) BOARD, REDUCE THE SCALAR SUM MOD 64

combined = board * numbered
total_sum = sum(sum(combined.values))
mod_sum = total_sum % 64
mod_sum

43

- the "current state" of the board is "encoded" as 43, the possibilities, due to the mod, are 0 through 63
- wherever the key is placed will be given the code of 0 through 63 on the "numbered" board, and that is the desired "end state"
- with this strategy, we can flip the coin on the numbered space that is the difference between the "current state" and the desired "end state"
- for example, the "current state" is 43. Say that the key is placed under space 57. We would flip the coin in space 14 from 0 to 1
- or "unflip" the coin in space 50
- this will increase the mod sum from 43 to 57
- this would communicate to prisoner 2, when he calculates that the mod sum is now 57, that the key is under that space


- However, this strategy only wins 75% of the time
- If the key is placed under space 59, we would need to flip 16 or "unflip" (from 1 to 0 or tails to heads) 48
- Since 16 is already "flipped" (tails), and 48 is already unflipped (heads), we lose
- 1/4th of the time, we will get 2 Trues, 1/4th we will get 2 Falses, and the other 1/2 will be split True and False. We only need one True though
- So only 1/4th of the time will each of the two coins be in the wrong 50/50 state for us, giving us 2 Falses
- Since the warden is allowed to know the plan, he will flip the coins so that this happens to us every time, and we lose

In [168]:
# CHANGING THE BOARD SO THAT IT CAN HAVE A MOD SUM OF 57

modsum1 = (mod_sum + 1*14) % 64 #flipping 14
modsum2 = (mod_sum - 1*50) % 64 #flipping 50
print(modsum1 == 57)
print(modsum2 == 57)

True
True


In [169]:
# CHANGING THE BOARD SO THAT IT CAN HAVE A MOD SUM OF 58

modsum3 = (mod_sum - 1*15) % 64 #flipping 15
modsum4 = (mod_sum - 1*49) % 64 #flipping 49
print(modsum3 == 58)
print(modsum4 == 58)

False
True


In [170]:
# TRYING TO CHANGE THE BOARD SO THAT IT CAN HAVE A MOD SUM OF 59
modsum5 = (mod_sum - 1*16) % 64 #flipping 16
modsum6 = (mod_sum + 1*48) % 64 #flipping 48
print(modsum5 == 59)
print(modsum6 == 59)

False
False


### Number 2: Binary Digits 

- On binary board, find where each binary digit is "on"
- The digits in binary go from right to left, and the 8 columns and rows are labeled 0 to 7
- The first digit occurs in columns 1,3,5,7
- The second digit occurs in columns 2,3,6,7
- The third digit occurs in columns 4,5,6,7
- The fourth digit occurs in rows 1,3,5,7
- The fifth digit occurs in rows 2,3,6,7
- The sixth digit occurs in rows 4,5,6,7
- Calculate the number of 1s (tails) mod 2 in each of these 6 regions to make a 6 digit binary number that will represent the "current state" of the board

In [258]:
# CALCULATE CURRENT STATE BY SUMMING MOD 2 THE NUMBER OF TAILS IN EACH REGION

first_digit = sum(sum(board.iloc[:,[1,3,5,7]].values)) % 2
second_digit = sum(sum(board.iloc[:,[2,3,6,7]].values)) % 2
third_digit = sum(sum(board.iloc[:,[4,5,6,7]].values)) % 2
fourth_digit = sum(sum(board.iloc[[1,3,5,7],:].values)) % 2
fifth_digit = sum(sum(board.iloc[[2,3,6,7],:].values)) % 2
sixth_digit = sum(sum(board.iloc[[4,5,6,7],:].values)) % 2

current_state = [sixth_digit,fifth_digit,fourth_digit,third_digit,second_digit,first_digit]
current_state

[1, 0, 0, 0, 1, 1]

- The "end state" must communicate where the key is, so it will be the binary conversion for the square number
- For example, if the key is on square 57, then the "end state" is 111001

In [260]:
# CALCULATE END STATE FOR SPACE 57

end_state = list((bin(57)[2:]))
end_state = list(map(int, end_state))
end_state

[1, 1, 1, 0, 0, 1]

- Now we must make some changes to get from the current state to the end state
- In this example, this means changing the second, fourth, and fifth digits
- This change is represented by 011010
- Square number 26 has that code, so we will flip the coin in that square
- Puzzle solved! This strategy has a 100% chance of success

## Solution Function

In [263]:
def find_coin(board, key:int):
    '''given a board setup (dataframe) and the key space (integer), find the right coin to flip'''
    
    #find current state
    first_digit = sum(sum(board.iloc[:,[1,3,5,7]].values)) % 2
    second_digit = sum(sum(board.iloc[:,[2,3,6,7]].values)) % 2
    third_digit = sum(sum(board.iloc[:,[4,5,6,7]].values)) % 2
    fourth_digit = sum(sum(board.iloc[[1,3,5,7],:].values)) % 2
    fifth_digit = sum(sum(board.iloc[[2,3,6,7],:].values)) % 2
    sixth_digit = sum(sum(board.iloc[[4,5,6,7],:].values)) % 2
    
    current_state = [sixth_digit,fifth_digit,fourth_digit,third_digit,second_digit,first_digit]
    
    #find end state, format it
    key_binary = list(map(int, list((bin(key)[2:]))))
    length = len(key_binary)
    end_state = list(map(int,np.zeros(6 - length)))
    for i in np.arange(length):
        end_state.append(key_binary[i])
    end_state
    
    #assert current state and end state are same length
    assert len(current_state) == len(end_state)
    
    #find necessary change
    change = []
    for i in np.arange(len(current_state)):
        if current_state[i] == end_state[i]:
            change.append(0)
        else:
            change.append(1)
    
    #convert binary to decimal
    def bin2dec(num_list): 
        dec_value = 0 
        base1 = 1
        len1 = len(num_list)
        for i in range(len1-1,-1,-1): 
            if num_list[i] == 1:      
                dec_value += base1 
            base1 = base1 * 2 
        return dec_value
    number = bin2dec(change)
    
    #return necessary coin flip
    return print('Flip coin',number)

In [264]:
find_coin(board, 57)

Flip coin 26
